<img src="data/images/lecture-notebook-header.png" />

# Data Stream Mining: Bloom Filter

In data mining, Bloom filters are probabilistic data structures used for efficient membership testing. They are designed to quickly determine whether an element is a member of a set or not, with a minimal amount of memory.

A Bloom filter consists of a bit array of a fixed size (typically a large number of bits) and a set of independent hash functions. When an element is added to the Bloom filter, it is hashed by each of the hash functions, and the corresponding bits in the bit array are set to 1. The more hash functions used, the lower the probability of false positives.

To check if an element is in the set, it is hashed by the same set of hash functions, and the corresponding bits in the Bloom filter are examined. If any of these bits are zero, then the element is definitely not in the set. However, if all the bits are one, the element is likely in the set, but there is a possibility of a false positive.

The key advantage of Bloom filters is their space efficiency. They require much less memory compared to storing the entire set of elements. However, there is a trade-off between space efficiency and accuracy. As the number of elements in the set increases, the probability of false positives also increases.

Bloom filters are commonly used in various data mining and information retrieval applications. They can be utilized to quickly filter out elements that are not in a set, reducing the need for expensive disk or network accesses. They are particularly useful in scenarios where the cost of false positives is acceptable, such as caching, spell checking, network routers, and web search engines.

## Setting up the Notebook

### Make all Required Imports

This notebook requires the [`bitarray`](https://pypi.org/project/bitarray/) package.

In [ ]:
import numpy as np
import pandas as pd

import hashlib
from bitarray import bitarray
from bitarray.util import ba2int, int2ba

### Create Set S

We use the simple example from the lecture, where we want to whitelist 5 email addresses for a spam filter.

In [ ]:
S = [
 'alice@gmail.com',
 'bob@yahoo.com',
 'chris@comp.nus.edu.sg',
 'dave@outlook.com',
 'erin@hotmail.com'
]

print(S)

---

## (Basic) Bloom Filter

In its core, a Bloom Filter is bit array `B` of a fixed size. Using $1..k$ hash function, an element is mapped to $k$ positions in the bit array -- in principle, it can be less than $k$ positions if one or more of the $k$ hash functions map to the same position.


### Define Hash Function

The following function maps an arbitrary string to an integer:

* `num_digits`: specifies the maximum number of digits and therefore the range of the possible hash values. For example, if `num_digits=3` the possible values are in `[0, 999]`.

* `nr`: A simple/naive way to simulate multiple hash functions. `nr` is just a factor the input string is multiple with to yield a different hash value. For example, `'Hello World'*3`, i.e., `nr=3`, results in `'Hello WorldHello WorldHello World'`

Note that the definition of "good" hash functions is an important subject on its own and beyond our scope here.

In [ ]:
def h(s, num_digits=1, nr=1):
    return int(hashlib.sha1((s*nr).encode("utf-8")).hexdigest(), 16) % (10 ** num_digits)

Let's quickly test method `h()` with different values for `num_digits` and `nr`.

In [ ]:
s = 'Hello World'

for num_digits in range(1, 4):
    for nr in range(1, 4):
        hash_value = h(s, num_digits=num_digits, nr=nr)
        
        print('Hash value for "{}" (num_digits={}, nr={}): {}'.format(s, num_digits, nr, hash_value))

### Initialize Bit Array B

Creating and initializing bit array B is straightforward, particularly when using the [`bitarray`](https://pypi.org/project/bitarray/) package.

In [ ]:
def init_B(num_buckets):
    B = bitarray(num_buckets, endian='big')
    B.setall(False)
    return B

B = init_B(10)

print('B = {}'.format(B))

Setting the elements of the bit array is as easy as setting the elements of a list.

In [ ]:
# Both 1 and True can be used to set a bit
B[3] = 1
B[9] = True

print('B = {}'.format(B))

### Filling B Using S

We define 2 helper methods to fill the bit array `B`. Method `add_key()` adds a single key `s` to `B`. To this end, the method applies `num_hash_functions` to the key and sets the respective positions in `B` to 1. Method `fill_B()` loops over all keys in `S` and calls `add_key()` to add each key to the bit array `B`.

In [ ]:
def add_key(s, B, num_digits, num_hash_functions):
    for nr in range(1, num_hash_functions+1):
        # Hash key to 0..10^{num_digits}
        pos = h(s, num_digits=num_digits, nr=nr)
        # Set the bit at position pos to 1
        B[pos] = 1


def fill_B(S, B, num_digits, num_hash_functions):
    # Loop over all keys s in S
    for s in S:
        add_key(s, B, num_digits, num_hash_functions)

With these 2 methods, we can add our set S of 5 email addresses to B. By default, we use `num_digits=2` which requires an bit array of size `10**num_digits`, as well as 3 hash functions. With `num_digits=2`, B will be still small enough to inspect it visually. But feel free to change the parameters.

In [ ]:
num_digits = 2
num_buckets = 10 ** num_digits
num_hash_functions = 3

B = init_B(num_buckets)
fill_B(S, B, num_digits, num_hash_functions)

print(B)

### Check/Lookup Keys

With B in place, we can now check if an element is in S. The method `check_key()` works very similar to `add_key()` by applying the `num_hash_functions` hash function to the element to get the respective positions in B. If *all* the bits in the respective positions are 1, the element is considered to be in set S.

In [ ]:
def check_key(s, B, num_digits, num_hash_functions):
    cnt = 0
    for nr in range(1, num_hash_functions+1):
        # Hash key to 0..10^{num_digits}
        pos = h(s, num_digits=num_digits, nr=nr)
        # Sum up the bit values at the respective positions
        cnt += int(B[pos])
    # Check if all positions are set to 1
    if cnt == num_hash_functions:
        return True
    else:
        return False

Let's test the method. You can change string `s` to any arbitrary string.

In [ ]:
s = 'erin@hotmail.com'
#s = 'scammer@hotmail.com'
        
found = check_key(s, B, num_digits, num_hash_functions)

if found == True:
    print('Key "{}" is (most likely!) in B'.format(s))
else:
    print('Key "{}" is definitely not in B'.format(s))

### False Positive Rate

We have seen in the lecture that Bloom Filters can result in false positives -- that is, a key that is not in S is "found" in B because it so happens that all hash functions map that key to bits that are set to 1. More precisely, the calculates as

$$
1 - e^{-\frac{|S|}{|B|}}
$$

Simply speaking, the error rate goes down if we increase the size of B or set S contains less elements. This is rather intuitive.

To get a better sense of this problem, we use a file containing ~100k dictionary English words and check if they are in S with respect to our Bloom Filter. By default, we use the same values for the parameters as above: `num_digits=2` and `num_hash_function=3`. None of those words are email addresses so we already know that none of those words should be found in B. Hence, every hit we can count as false positive.

In [ ]:
%%time

num_digits = 2
num_buckets = 10 ** num_digits
num_hash_functions = 3

B = init_B(num_buckets)
fill_B(S, B, num_digits, num_hash_functions)


num_false_positives = 0

with open('data/datasets/language/english-lowercase.txt') as f:
    for line in f:
        word = line.strip()
        if check_key(word, B, num_digits, num_hash_functions) == True:
            num_false_positives += 1

            
print('Number of false positives: {}'.format(num_false_positives))

For `num_digits=2` and `num_hash_function=3` we get indeed 199 false positives. However, once we increase the size of `B` and thus decrease the probability of false positives, no dictionary word will be found in B.

Of course, in this example, our set S was extremely small so a bit array with 1000 bits is already sufficient. In practice, Bloom Filters can easily have a size of several billion bits, since 8 billion bits translate to 1 GB which can easily be kept in memory on most machines.

## Counting Bloom Filter

As you already know, a basic Bloom filter does not support the removal of items in S. If keys get removed from S, in principle, the bit array as to be re-created from scratch. The issues is that we cannot simply set bits from 1 back to zero when deleting a key, since a 1 bit could haven been set by multiple keys.

A solution to this problem are Counting Bloom Filter which replace a single bit in the array with multiple bits which in themselves represent a counter. For example, using 3 bits, one can represent a count from 0 to 7. Of curse, a a trade off the overall size of the bit arrays increases by a factor of 3


### Bit Array B

Initializing B for a Counting Bloom Filter is still straightforward; we only need to extend its size accordingnumber of bits required to represent each counter in B.

In [ ]:
def init_counting_B(num_buckets, bits_per_counter):
    B = bitarray(num_buckets*bits_per_counter)
    B.setall(False)
    return B

B = init_counting_B(10, 3)

print('B = {}'.format(B))

### Updating a Counter

Each time we want to add or delete a key, we have to update all respective counters in B -- that is, the counters that are identified by the different hash functions. Each update either increases or increases a counter by 1. The method `update_counter()` implements this.

Recall that the counter might overflow or underflow (e.g., if a key gets deleted that never was added). Both cases are pathological and we simply raise an exception here.

In [ ]:
def update_counter(counter, bits_per_counter, update='inc'):
    
    # Convert bit array to integer value
    int_val = ba2int(counter)
    
    # Depending on the update method, increase or decrease the integer value by one
    if update == 'inc':
        int_val += 1
    else:
        int_val -= 1
    
    # If the integer value is negative, raise an exception
    if int_val < 0:
        raise Exception('Error: bit counter < 0')
        
    # If the counter overflows, raise an axception
    if int_val >= 2**bits_per_counter:
        raise Exception('Error: counter larger than max size')
        
    # Convert integer value to a bit string, adding leading zeros if needed
    bit_string = format(int_val, '0{}b'.format(bits_per_counter))
    
    # return new counter as bitarray object
    return bitarray(bit_string)

### Updating B Given a Key s

Since inserting and deleting keys require more or less the same steps, we define a method `update_key()` to reduce all counters for a given key `s`. The methods `insert_key()` and `delete_key()` simply call method `update_key` with the respective parameters.

In [ ]:
def update_key(s, B, num_digits, num_hash_functions, bits_per_counter, update):
    for nr in range(1, num_hash_functions+1):
        # Hash key to 0..10^{num_digits}
        pos = h(s, num_digits=num_digits, nr=nr)
        # Get the counter from B
        counter = B[pos*bits_per_counter:(pos*bits_per_counter+bits_per_counter)]
        # update counter
        if update == 'insert':
            counter = update_counter(counter, bits_per_counter, update='inc')
        else:
            counter = update_counter(counter, bits_per_counter, update='dec')
        # Set updated counter in B
        B[pos*bits_per_counter:(pos*bits_per_counter+bits_per_counter)] = counter
        
        
def insert_key(s, B, num_digits, num_hash_functions, bits_per_counter):
    update_key(s, B, num_digits, num_hash_functions, bits_per_counter, 'insert')

    
def delete_key(s, B, num_digits, num_hash_functions, bits_per_counter):
    update_key(s, B, num_digits, num_hash_functions, bits_per_counter, 'delete')

#### Test: Insert Keys

As a simple test, we try to insert the same key `s` 10 times. This helps us to visualize what's going on, as well as to see if the exception is raised if a counter overflows.

In [ ]:
num_digits = 1
num_buckets = 10 ** num_digits
num_hash_functions = 3
bits_per_counter = 3

B = init_counting_B(num_buckets, bits_per_counter)

s = 'Hello World'

for _ in range(10):
    try:
        insert_key(s, B, num_digits, num_hash_functions, bits_per_counter)
        print(B)
    except Exception as e:
        print(e)

Since we use 3 hash functions, we can see the 3 individual counters increasing after each insert. When we try to insert the same key `s` for the 8th time, we get an error since a counter of 3 bits can no longer represent this count. That means our check for overflows works correctly.

#### Test: Delete  Keys

In the test below, we first add a key `s` once and then try to delete it twice. While the first delete should result in a bit array with all bits set to zero again, the second attempt to delete `s` should throw an error.

In [ ]:
num_digits = 1
num_buckets = 10 ** num_digits
num_hash_functions = 3
bits_per_counter = 3

B = init_counting_B(num_buckets, bits_per_counter)

s = 'Hello World'

try:
    insert_key(s, B, num_digits, num_hash_functions, bits_per_counter)
    print(B)
    delete_key(s, B, num_digits, num_hash_functions, bits_per_counter)
    print(B)
    delete_key(s, B, num_digits, num_hash_functions, bits_per_counter)
    print(B)
except Exception as e:
    print(e)

As expected, the insert and the first delete work just fine. Trying to delete `s` a second time would result in an underflow of the counter. We therefore correctly see the exception raised.

### Creating B

Now we can create `B` with our example set `S` containing the 5 email addresses by calling `insert_key` for each email address in `S`.

In [ ]:
num_digits = 2
num_buckets = 10 ** num_digits
num_hash_functions = 3
bits_per_counter = 3

B = init_counting_B(num_buckets, bits_per_counter)

for s in S:
    insert_key(s, B, num_digits, num_hash_functions, bits_per_counter)

print(B)

### Check/Lookup Keys

The lookup of keys is pretty much identical to the basic Bloom Filters. We only need to check if all counters -- identified by the hash functions -- are greater than zero.

In [ ]:
def check_key_counting(s, B, num_digits, num_hash_functions):
    cnt = 0
    for nr in range(1, num_hash_functions+1):
        # Hash key to 0..10^{num_digits}
        pos = h(s, num_digits=num_digits, nr=nr)
        
        # Get counter (i.e. the respective bits)
        counter = B[pos*bits_per_counter:(pos*bits_per_counter+bits_per_counter)]
        
        # Convert counter bit array to integer and check if it is larger than 0
        if ba2int(counter) > 0:
            cnt += 1
        
    # Check if all positions are set to 1
    if cnt == num_hash_functions:
        return True
    else:
        return False

#### Test: Check for Presence of a Key s in B

Given our bit array B, we can now check if a key `s` can be found in `B`. By default, `s` is an email address we know is in S, so the result should be positive. Feel free to change `s` and check the output.

In [ ]:
s = 'erin@hotmail.com'
#s = 'scammer@hotmail.com'
        
found = check_key_counting(s, B, num_digits, num_hash_functions)

if found == True:
    print('Key "{}" is (most likely!) in B'.format(s))
else:
    print('Key "{}" is definitely not in B'.format(s))

#### Test: Check for Presence of a Key s in B after Deleting it

Since we now have a Counting Bloom Filter, we can now delete keys. The following example shows the removal of a key and the lookup of that key after the deletion. Of course, this lookup should fail.

In [ ]:
s = 'erin@hotmail.com'

delete_key(s, B, num_digits, num_hash_functions, bits_per_counter)

found = check_key_counting(s, B, num_digits, num_hash_functions)

if found == True:
    print('Key "{}" is (most likely!) in B'.format(s))
else:
    print('Key "{}" is definitely not in B'.format(s))


**Important:** This implementation of a Counting Bloom Filter is rather simplistic. For example, when delete a key, the method `update_key()` does not check if updating all counters were successful. For example, it can happen that decreasing the counter with respect to the first 2 hash functions will work but fail for the 3rd hash function. This should raise an exception and should not decrease any of the 3 counters.

---

## Summary

Bloom filters are probabilistic data structures used for efficient membership testing in data mining and information retrieval applications. They provide a space-efficient solution for determining whether an element is a member of a set or not, using a minimal amount of memory.

The key advantage of Bloom filters is their compactness. They require significantly less memory compared to storing the entire set of elements. This makes them suitable for scenarios where memory usage is a concern. Additionally, Bloom filters offer constant-time complexity for insertion and membership testing operations, regardless of the size of the set.

However, Bloom filters come with some limitations. One notable drawback is the potential for false positives. Due to the probabilistic nature of the data structure, there is a small probability of mistakenly indicating that an element is in the set when it is not. The probability of false positives increases as the number of elements in the set or the occupancy of the Bloom filter increases.

Another limitation is that at least Bloom filters do not support element deletion. Once an element is inserted, it cannot be removed from the filter without affecting the accuracy of other elements. Therefore, Bloom filters are typically used for static sets or applications where insertions are frequent but deletions are not required. While Counting Bloom filters support element deletion, they come with the trade-off of increased memory requirements (and some additional computing requirements).

In summary, Bloom filters offer a space-efficient solution for membership testing by utilizing a compact bit array and independent hash functions. They provide fast insertion and lookup operations with a low memory footprint. However, the trade-off is a possibility of false positives and the inability to delete elements. Bloom filters are well-suited for scenarios where memory efficiency is crucial, and a small rate of false positives is acceptable.